<a href="https://colab.research.google.com/github/Komal-Zia/PAIassignment03/blob/main/AIAssignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch scikit-learn


In [ ]:
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd


In [ ]:
# Load datasets
train_df = pd.read_csv('/content/train_binary_sent.csv')
dev_df = pd.read_csv('/content/dev_binary_sent.csv')
test_df = pd.read_csv('/content/test_binary_sent.csv')

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a custom dataset
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
max_len = 128
train_dataset = SentimentDataset(train_df['sentence'], train_df['label'], tokenizer, max_len)
dev_dataset = SentimentDataset(dev_df['sentence'], dev_df['label'], tokenizer, max_len)
test_dataset = SentimentDataset(test_df['sentence'], test_df['label'], tokenizer, max_len)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
#Define RNN_BERT Model
class RNNBERTModel(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(RNNBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.rnn = nn.LSTM(
            input_size=self.bert.config.hidden_size,
            hidden_size=hidden_size,
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Bidirectional

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze BERT weights
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        rnn_output, _ = self.rnn(bert_output.last_hidden_state)
        output = self.fc(rnn_output[:, 0, :])  # Use the output of the first token ([CLS])
        return output


In [ ]:
#Training Function
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


In [ ]:
#Evaluation Function
def evaluate_model(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    return accuracy, precision, recall, f1


In [ ]:
# Initialize model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNBERTModel(hidden_size=256, num_classes=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Train and evaluate the model
epochs = 3
for epoch in range(epochs):
    train_loss = train_model(model, train_loader, optimizer, criterion, device)
    dev_accuracy, dev_precision, dev_recall, dev_f1 = evaluate_model(model, dev_loader, device)
    print(f"Epoch {epoch + 1}: Train Loss = {train_loss:.4f}, Dev Accuracy = {dev_accuracy:.4f}, "
          f"Precision = {dev_precision:.4f}, Recall = {dev_recall:.4f}, F1 Score = {dev_f1:.4f}")


Epoch 1: Train Loss = 0.5980, Dev Accuracy = 0.8268, Precision = 0.8097, Recall = 0.8626, F1 Score = 0.8353
Epoch 2: Train Loss = 0.3799, Dev Accuracy = 0.8498, Precision = 0.8366, Recall = 0.8761, F1 Score = 0.8559
Epoch 3: Train Loss = 0.3449, Dev Accuracy = 0.8475, Precision = 0.8104, Recall = 0.9144, F1 Score = 0.8593


In [23]:
# Test the model
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(model, test_loader, device)
print(f"Test Results: Accuracy = {test_accuracy:.4f}, Precision = {test_precision:.4f}, "
      f"Recall = {test_recall:.4f}, F1 Score = {test_f1:.4f}")


Test Results: Accuracy = 0.8402, Precision = 0.8059, Recall = 0.8955, F1 Score = 0.8484
